In [1]:
pip install keras-tuner

     |████████████████████████████████| 71kB 3.7MB/s 
  Created wheel for keras-tuner: filename=keras_tuner-1.0.2-cp36-none-any.whl size=78939 sha256=cd09c43355d7663b2ae5052a390ce04bb4700d4880f845431ff98ac52ee62d93
  Stored in directory: /root/.cache/pip/wheels/bb/a1/8a/7c3de0efb3707a1701b36ebbfdbc4e67aedf6d4943a1f463d6
  Created wheel for terminaltables: filename=terminaltables-3.1.0-cp36-none-any.whl size=15358 sha256=bfd8d190286da995bbd68d4d67560c0762049afad97a30a0b31d9f54e8457d99
  Stored in directory: /root/.cache/pip/wheels/30/6b/50/6c75775b681fb36cdfac7f19799888ef9d8813aff9e379663e
Successfully built keras-tuner terminaltables


In [1]:
import zipfile
import os
import cv2
import shutil

import numpy as np
import pandas as pd
from datetime import datetime as dt
from matplotlib import pyplot as plt
import json
import re
import pickle

from sklearn.model_selection import train_test_split

from tensorflow.keras import layers
from tensorflow.keras import Model
from tensorflow.keras.preprocessing import image 
from tensorflow.keras import backend as K
from tensorflow.keras import applications
from tensorflow.keras import utils
from tensorflow.keras import regularizers
from tensorflow.keras import optimizers
from tensorflow.keras import callbacks
from tensorflow.keras.models import load_model
from tensorflow.keras import callbacks

#import kerastuner

Importazione dati

In [99]:
shutil.copyfile(
    '/content/drive/MyDrive/DSIM/image_classification/imagenet_mini_transformed.zip',
    'imagenet_mini_transformed.zip')

image_data = zipfile.ZipFile('imagenet_mini_transformed.zip')
image_data.extractall()
image_data.close()

In [4]:
shutil.copyfile('/content/drive/MyDrive/DSIM/image_classification/archive.zip', 'archive.zip')

image_data = zipfile.ZipFile('archive.zip')
image_data.extractall()
image_data.close()

Unione dei due dataset

In [100]:
for folder in ['motion_blurred', 'defocused_blurred', 'sharp']:
  for f in os.listdir(folder):
    shutil.copyfile(folder + '/' + f,
                    'imagenet_mini_transformed/' + folder + '/' + f)

In [2]:
def build_model(pre_trained, cut, optimizer):
  
  # elenco dei nomi dei layers
  layer_names = [layer.name for layer in pre_trained.layers]
  layer_idx = layer_names.index(cut) + 1
  
  # ricostruzione architettura dal primo layer non congelato (layer_idx)
  inputs = layers.Input(X_train.shape[1:])
  x = inputs
  for layer in pre_trained.layers[layer_idx:]:
    x = layer (x)
  
  # aggiunta layers di dropout e pooling
  x = layers.GlobalAveragePooling2D() (x)
  x = layers.Dropout(0.5)(x)

  # output layer
  x = layers.Dense(3, activation='softmax')(x)

  model = Model(inputs = inputs, outputs = x)
  
  model.compile(
      loss = 'categorical_crossentropy',
      optimizer = optimizer,
      metrics = ["accuracy"]
      )
  
  return model

In [3]:
# funzione per visualizzazione immagini
def display_img(img_list, nrow = 1, ncol = 0, size = (10,10), text = []):

  if not ncol:
    ncol = len(img_list)
  
  fig = plt.figure(figsize=(size[0], size[1]))
  k = 0
  for i in range(nrow):
    for j in range(ncol):
      current_plot = fig.add_subplot(nrow, ncol, k+1)
      if k < len(text):
        current_plot.title.set_text(text[k])
      plt.imshow(img_list[k])
      k += 1

Data loader e preprocessing

In [4]:
processing = image.ImageDataGenerator(
      preprocessing_function = applications.mobilenet.preprocess_input,
      validation_split = 0.3
      )

train_generator = processing.flow_from_directory(
      directory = 'imagenet_mini_transformed',
      target_size = (224, 224), 
      color_mode = 'rgb', 
      batch_size = 256,
      class_mode = 'categorical',
      shuffle = False,
      subset = 'training',
      seed = 3
      )
valid_generator = processing.flow_from_directory(
      directory = 'imagenet_mini_transformed',
      target_size = (224, 224), 
      color_mode = 'rgb', 
      batch_size = 256,
      class_mode = 'categorical',
      shuffle = False,
      subset = 'validation',
      seed = 3
      )

Found 7665 images belonging to 3 classes.
Found 3285 images belonging to 3 classes.


Mobilenet pre-addestrata

In [6]:
K.clear_session()
mobilenet = applications.MobileNet((224,224,3),include_top=False)
cut = 'conv_dw_5_relu'
mobilenet_pre_trained = Model(mobilenet.input, outputs = mobilenet.get_layer(cut).output)

Limitazione nella rappresentazione dei float per evitare errori per OOM

In [5]:
K.set_floatx('float16')

Estrazione output ultimo layer congelato

In [7]:
# mescolo le obs di training
train_obs = len(train_generator.filenames)
shuffle_train = np.random.RandomState(seed = 42).permutation(train_obs)

X_train = mobilenet_pre_trained.predict(train_generator)[shuffle_train]
X_val = mobilenet_pre_trained.predict(valid_generator)

Target labels

In [8]:
# output labels
y_train = utils.to_categorical(np.reshape(train_generator.classes[shuffle_train], -1), 3)
y_valid = utils.to_categorical(np.reshape(valid_generator.classes, -1), 3)

Addestramento con optimizer adam

In [9]:
base_model = build_model(
    mobilenet,
    cut = cut,
    optimizer = 'adam'
    )
base_model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 28, 28, 256)]     0         
_________________________________________________________________
conv_pw_5 (Conv2D)           (None, 28, 28, 256)       65536     
_________________________________________________________________
conv_pw_5_bn (BatchNormaliza (None, 28, 28, 256)       1024      
_________________________________________________________________
conv_pw_5_relu (ReLU)        (None, 28, 28, 256)       0         
_________________________________________________________________
conv_pad_6 (ZeroPadding2D)   (None, 29, 29, 256)       0         
_________________________________________________________________
conv_dw_6 (DepthwiseConv2D)  (None, 14, 14, 256)       2304      
_________________________________________________________________
conv_dw_6_bn (BatchNormaliza (None, 14, 14, 256)       1024

In [10]:
base_model.fit(
    X_train, y_train,
    validation_data=(X_val, y_valid),
    epochs = 20,
    batch_size = 32,
    callbacks = [callbacks.ModelCheckpoint(
        filepath = 'ckp_',
        monitor = "val_accuracy",
        verbose = 0,
        save_best_only = True,
        save_weights_only = False
        )]
    )

Epoch 1/20
240/240 [==============================] - 37s 132ms/step - loss: 0.5143 - accuracy: 0.8761 - val_loss: 17.2812 - val_accuracy: 0.4607
INFO:tensorflow:Assets written to: ckp_/assets
Epoch 2/20
240/240 [==============================] - 26s 109ms/step - loss: 0.1504 - accuracy: 0.9500 - val_loss: 1.4082 - val_accuracy: 0.7930
INFO:tensorflow:Assets written to: ckp_/assets
Epoch 3/20
240/240 [==============================] - 26s 109ms/step - loss: 0.1103 - accuracy: 0.9657 - val_loss: 0.9536 - val_accuracy: 0.7739
Epoch 4/20
240/240 [==============================] - 26s 109ms/step - loss: 0.1095 - accuracy: 0.9678 - val_loss: 5.4922 - val_accuracy: 0.7441
Epoch 5/20
240/240 [==============================] - 26s 109ms/step - loss: 0.0813 - accuracy: 0.9708 - val_loss: 3.9512 - val_accuracy: 0.7529
Epoch 6/20
240/240 [==============================] - 26s 109ms/step - loss: 0.0699 - accuracy: 0.9831 - val_loss: 1.3340 - val_accuracy: 0.8062
INFO:tensorflow:Assets written to: 

In [17]:
# memorizzazione checkpoints
shutil.make_archive('ckp_adam', 'zip', 'ckp_adam')
shutil.copyfile('ckp_adam.zip', '/content/drive/MyDrive/DSIM/image_classification/ckp_adam.zip')

'/content/drive/MyDrive/DSIM/image_classification/ckp_adam.zip'

Addestramento con optimizer AdaMax

In [14]:
base_model_adamax = build_model(
    mobilenet,
    cut = cut,
    optimizer = 'adamax'
    )
base_model_adamax.summary()

Model: "model_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         [(None, 28, 28, 256)]     0         
_________________________________________________________________
conv_pw_5 (Conv2D)           (None, 28, 28, 256)       65536     
_________________________________________________________________
conv_pw_5_bn (BatchNormaliza (None, 28, 28, 256)       1024      
_________________________________________________________________
conv_pw_5_relu (ReLU)        (None, 28, 28, 256)       0         
_________________________________________________________________
conv_pad_6 (ZeroPadding2D)   (None, 29, 29, 256)       0         
_________________________________________________________________
conv_dw_6 (DepthwiseConv2D)  (None, 14, 14, 256)       2304      
_________________________________________________________________
conv_dw_6_bn (BatchNormaliza (None, 14, 14, 256)       1024

In [15]:
base_model_adamax.fit(
    X_train, y_train,
    validation_data=(X_val, y_valid),
    epochs = 20,
    batch_size = 32,
    callbacks = [callbacks.ModelCheckpoint(
        filepath = 'ckp_adamax',
        monitor = "val_accuracy",
        verbose = 0,
        save_best_only = True,
        save_weights_only = False
        )]
    )

Epoch 1/20
240/240 [==============================] - 29s 112ms/step - loss: 0.2501 - accuracy: 0.9190 - val_loss: 0.6997 - val_accuracy: 0.8320
INFO:tensorflow:Assets written to: ckp_adamax/assets
Epoch 2/20
240/240 [==============================] - 26s 109ms/step - loss: 0.0457 - accuracy: 0.9913 - val_loss: 0.7666 - val_accuracy: 0.8306
Epoch 3/20
240/240 [==============================] - 26s 108ms/step - loss: 0.0357 - accuracy: 0.9928 - val_loss: 0.9019 - val_accuracy: 0.8296
Epoch 4/20
240/240 [==============================] - 26s 108ms/step - loss: 0.0331 - accuracy: 0.9903 - val_loss: 1.0889 - val_accuracy: 0.8198
Epoch 5/20
240/240 [==============================] - 26s 109ms/step - loss: 0.0351 - accuracy: 0.9906 - val_loss: 1.0508 - val_accuracy: 0.8208
Epoch 6/20
240/240 [==============================] - 26s 108ms/step - loss: 0.0343 - accuracy: 0.9912 - val_loss: 0.8848 - val_accuracy: 0.8354
INFO:tensorflow:Assets written to: ckp_adamax/assets
Epoch 7/20
240/240 [====

In [19]:
# memorizzazione checkpoints
shutil.make_archive('ckp_adamax', 'zip', 'ckp_adamax')
shutil.copyfile('ckp_adamax.zip', '/content/drive/MyDrive/DSIM/image_classification/ckp_adamax.zip')

'/content/drive/MyDrive/DSIM/image_classification/ckp_adamax.zip'

Le performance sembrano essere meno soddisfacenti della CNN addestrata sul dataset originale